In [1]:
import re
from bigdl.util.common import * 
from bigdl.transform.vision.image import *
from bigdl.transform.vision import image
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, StringType
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from pyspark import SparkConf
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from zoo.common.nncontext import *
from zoo.pipeline.nnframes.nn_classifier import *
from zoo.pipeline.nnframes.nn_image_reader import *
from zoo.pipeline.nnframes.nn_image_transformer import *

creating: createDefault
creating: createSGD


In [2]:
sparkConf = SparkConf().setAppName("ImageTransferLearningExample")
sc = get_nncontext(sparkConf)
sc

<SparkContext master=local[2] appName=PySparkShell>

In [3]:
model_path = '../../../model/bigdl_inception-v1_imagenet_0.4.0.model' 
preTrainedNNModel = NNModel(Model.loadModel(model_path), [3,224,224]).setPredictionCol("embedding")

creating: createNNModel


## Get Flowers

In [4]:
!curl -LO http://download.tensorflow.org/example_images/flower_photos.tgz
!tar xzf flower_photos.tgz
!rm flower_photos/LICENSE.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  69.9M      0  0:00:03  0:00:03 --:--:-- 69.9M


In [5]:
def flowers():
    classes = {}
    file_name_set = set()
    file_name_class = {}

    image_path = 'flower_photos'
    for dir_name in os.listdir(image_path):
        if os.path.isdir(image_path + '/' + dir_name):
            print(dir_name)
            classes.setdefault(dir_name, float(len(classes) + 1))
            for file_name in os.listdir(image_path + '/' + dir_name):
                if file_name in file_name_set:
                    print('Duplicate file name', file_name)
                file_name_set.add(file_name)
                file_name_class[file_name] = classes[dir_name]
            
    return classes, file_name_class

classes, train_image_to_class = flowers()

tulips
roses
('Duplicate file name', '15922772266_1167a06620.jpg')
daisy
sunflowers
dandelion


In [6]:
classes

{'daisy': 3.0,
 'dandelion': 5.0,
 'roses': 2.0,
 'sunflowers': 4.0,
 'tulips': 1.0}

## Read files

In [ ]:
data_path = 'flower_photos/*/*'
image_path = data_path
imageDF = NNImageReader.readImages(image_path, sc).repartition(12).cache()
print ("partition number: ", imageDF.rdd.getNumPartitions())
print ("image number: ", imageDF.count())

('partition number: ', 12)


In [ ]:
# os.listdir(data_path + '/train_img')

In [ ]:
imageDF.show()

#### Create array of labels

In [ ]:
getName = udf(lambda row: row[0].split('/')[-1])
getLabel = udf(lambda name: train_image_to_class[name], DoubleType())
labelDF = imageDF \
    .withColumn("name", getName(col("image"))) \
    .withColumn("label", getLabel(col('name')))

In [ ]:
labelDF.show(20)

In [ ]:
(trainingDF, validationDF) = labelDF.randomSplit([0.9, 0.1])

In [ ]:
trainingDF.count()

Compose a pipeline

In [ ]:
transformer = NNImageTransformer(
    image.Pipeline([Resize(256, 256), 
                    CenterCrop(224, 224), 
                    ChannelNormalize(123.0, 117.0, 104.0)])). \
        setInputCol("image"). \
        setOutputCol("features")

In [ ]:
features = transformer.transform(trainingDF)

In [ ]:
trainingEmbedDF = preTrainedNNModel.transform(features)

In [ ]:
trainingEmbedDF.show()

Load pretrained model:

In [ ]:
EPOCHS = 100
LEARNING_RATE = 0.001
BATCH_SIZE = 64

Create Linear Classifier

In [ ]:
from bigdl.optim.optimizer import Optimizer, Adam, MaxEpoch, EveryEpoch, Top1Accuracy, \
    TrainSummary, ValidationSummary, SeveralIteration, SGD

In [ ]:
lrModel = Sequential().add(Linear(1000, len(classes))).add(LogSoftMax())

classifier = NNClassifier(lrModel, ClassNLLCriterion(), [1000]) \
        .setOptimMethod(SGD(nesterov=True, momentum=0.9, dampening=0.0)) \
        .setLearningRate(LEARNING_RATE) \
        .setBatchSize(BATCH_SIZE) \
        .setMaxEpoch(EPOCHS) \
        .setFeaturesCol("embedding")

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="accuracy")

In [ ]:
# pipeline = Pipeline(stages=[transformer, preTrainedNNModel, classifier])
# pipeline = Pipeline(stages=[transformer, preTrainedNNModel, classifier])

In [ ]:
grocery_model = classifier.fit(trainingEmbedDF)

In [ ]:
# trainPredictDF = grocery_model.transform(trainingDF)
# evaluator.evaluate(trainPredictDF)

In [ ]:
validationEmbedDF = preTrainedNNModel.transform(transformer.transform(validationDF))
predictionDF = grocery_model.transform(validationEmbedDF).cache()
predictionDF.show()

In [ ]:
evaluator.evaluate(predictionDF)